# Créer et exporter une catégorie au format Moodle XML

Besoin de 
`pip install array-to-latex` et `pip install xmltodict`

Si vous ne savez pas comment faire, décommentez la case suivante et exécutez le code (prend une bonne minute), ça devrait marcher

In [ ]:
#import sys
#!{sys.executable} -m pip install xmltodict
#!{sys.executable} -m pip install array-to-latex

In [44]:
# Permet a une cellule d'avoir plus d'un display en sortie
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Technical functions:

In [45]:
from xml.dom.minidom import parseString
from xml.sax.saxutils import unescape
import xmltodict
import json
import io

In [46]:
# to deal with mess between Latex, python and xml special characters
# \u and \x not supported but useless for inline latex?
UNESCAPE_LATEX = { '\x07':'\\a', '\x0c':'\\f', '\x0b':'\\v', '\x08':'\\b', '\n': '\\n', '\r':'\\r', '\t':'\\t' } 

In [47]:
def savestr(string, filename="new.txt", raw=False):
    if raw:
        string = string.replace('\t','') # no tabs
        string = string.replace('\n','') # no linebreak
    else:
        string = string.replace('\t','  ') # double space instead of tabs
    text_file = io.open(filename, "w", encoding='utf8') # essential for accents and other characters
    text_file.write(string)
    text_file.close()

def latex_protect(string):
    return unescape(string, UNESCAPE_LATEX)
    
def html(string):
    if string is "":
        return string
    else:
        return "<![CDATA[<p>\(\)" + latex_protect(string) + "</p>]]>"  # \(\) pour activer latex dans Moodle

In [48]:
def set_oparg(name, default_value, **opargs): #optional argument manager
    if name in opargs:
        return opargs.get(name)
    else:
        return default_value

### Création de Moodle XML

Global constants

In [112]:
# Dirty but needed : all the fields required to create a question
# We also set their default value, and an alias when the fields has a weird/toolong name
DICT_DEFAULT_QUESTION_MOODLE = { 
    # general stuff
    '@type': {'default': "essay", 
              'alias': 'type'
             }, # can be "multichoice" for MCQs
    "name": {'default': "Default question title", 
             'attribute': {'@format': 'txt'}, 
             'alias': 'title'
            },
    "questiontext": {'default': "Default question text", 
                     'attribute': {'@format': 'html'}, 
                     'alias': 'text'
                    },
    "generalfeedback": {'default': "", 'attribute': {'@format': 'html'}},
    "defaultgrade": {'default': 1.0, 
                     'alias': 'grade'
                    },
    "penalty": {'default': 0.0},
    "hidden": {'default': 0},
    "idnumber": {'default': ""},
    # 'essay' specifics
    "responseformat": {'default': "editorfilepicker"}, # by default allow to upload a file as answer. Set "editor" ottherwise
    "responserequired": {'default': 0}, # 0 for no response required, 1 for yes
    "responsefieldlines": {'default': 10},
    "attachments": {'default': -1}, # number of attachments allowed. -1 is infinty
    "attachmentsrequired": {'default': 0}, # 0 for no attachment required, 1 for yes
    "graderinfo": {'default': "", # correction for the grader
                   'attribute': {'@format': 'html'}, 
                   'alias': 'infocorrecteur'
                  }, 
    "responsetemplate": {'default': "", 'attribute': {'@format': 'html'}},
    # 'multichoice' specifics
    "single" : {'default': "true"}, # Says if only a unique answer is possible
    "shuffleanswers" : {'default': "true"}, # Constantly shuffles the possible choices
    "answernumbering" : {'default': "none"}, # Other choices : 'abc', '123', 'iii', and certainly caps
    "correctfeedback": {'default': "Votre réponse est correcte.", 'attribute': {'@format': 'html'}},
    "partiallycorrectfeedback": {'default': "Votre réponse est partiellement correcte.", 'attribute': {'@format': 'html'}},
    "incorrectfeedback": {'default': "Votre réponse est incorrecte.", 'attribute': {'@format': 'html'}},
    "shownumcorrect" : {'default': ""}, # No idea
    "answer" : {'default': ""} # We deal with this in the Answer class
}

# easy access to alias
def alias(field):
    if 'alias' in DICT_DEFAULT_QUESTION_MOODLE[field]:
        return DICT_DEFAULT_QUESTION_MOODLE[field]['alias']
    else:
        return field

Classes for Category, Question, Answer

In [50]:
class Answer():
    """ 
        Object collecting an answer to a multichoice Question
    """
    def __init__(self, answer_text="This is a default answer", grade=0):
        # we manage the default value of grade
        # grade can be either a int/float (percentage of the grade) or a bool (is the answer true or not)
        if isinstance(grade, bool) or isinstance(grade, np.bool):
            if grade:
                grade = 100
            else:
                grade = 0
        # otherwise it is a number we leave it as it is
        self.dict = {
            '@fraction': grade, # by default an answer is false, and gives no points
            '@format': 'html',
            'text': html(answer_text), # content of the answer
            'feedback': {
                '@format': 'html',
                'text': ""
            }
        }
    
    def text(self, text):
        self.dict['text'] = html(text)
        
    def feedback(self, text):
        self.dict['feedback']['text'] = html(text)
        
    def relativegrade(self, answer_fraction):
        self.dict['@fraction'] = answer_fraction # must be a number (int?) between 0 and 100, decribing how much it is worth
    
    def istrue(self): # Says that this answer is THE good one
        self.relativegrade(100)
        
    def isfalse(self): # Says that this answer is THE good one
        self.relativegrade(0)
        
    def addto(self, question): # includes the answer into a question
        """"""
        if question.dict['@type'] is "multichoice":
            if question.dict["answer"] is "": # if it is the first question we add
                question.dict["answer"] = []
            question.dict["answer"].append(self.dict)
        else:
            print('Error : answers can only be added to multichoice questions')
                     

In [107]:
class Question():
    """ 
        Object collecting the parameters of a question under the form of a dictionary.
        Methods:
        _set(field, value) : e.g. _set("questiontext", "What is $2+2$?")
    """
    def __init__(self, question_type="essay"):
        self.structure = DICT_DEFAULT_QUESTION_MOODLE # IMPORTANT
        self.structure['@type']['default'] = question_type # "essay" by default
        # The proper question in a dictionary ready to turn into xml
        self.dict = {}
        for field in self.structure:
            self._set(field, self.structure[field]['default'])
               
    def _set(self, field, value=""):
        """ Assigns a value to a field of a Question """
        field_structure = self.structure[field]
        if 'attribute' not in field_structure: # no attributes, just stupid value to assign 
            self.dict[field] = value
        else: # we have attributes which means the field contains a <text> element
            if 'html' in field_structure['attribute'].values(): # the value is a string to be turned...
                value = html(value)                             # ... into a html string (tackles latex, <p>'s and stuff)
            # now we just fill the field with a text element, and its attributes
            self.dict[field] = {**field_structure['attribute'], **{"text": value}} # concatenation needs Python >= 3.5
    
    def multi_answer(self): # unlocks the multiple answer mode
        self.dict["single"] = "false" #TBA : check sum fractions is 100 or all 0 etc
        
    def addto(self, category):
        category.append(self)

# Here we define automatically methods to assign values to Question fields
for key in DICT_DEFAULT_QUESTION_MOODLE.keys():
    if key is not "answer": # Because could be misinterpreted with Question.dict["answer"]
        setattr(Question, alias(key), lambda self, value, key=key: self._set(key, value))

In [8]:
class Category():
    """ 
        Object collecting Questions under the form of a category, ready to export to Moodle.
        Methods:
        _set(name, description) : e.g. _set("my_category", "list of questions about ... ")
        append(question) : adds a Question to the Category
        save(file_name) : save the Category into Moodle-XML
    """
    def __init__(self, name="Default category name", description=""):
        self.dict = { "quiz": { "question": [{}] } }
        self.questions = self.dict["quiz"]["question"]
        self._set(name, description)
    
    def _set(self, name="Default category name", description=""):
        qcat = {
            "@type": "category",
            "category": {"text": "$module$/top/" + name},
            "info": {"@format": "html", "text": html(description)}
            }
        self.questions[0] = qcat
        
    def name(self, string="Default category name"):
        self.questions[0]['category']['text'] = "$module$/top/" + string
        
    def description(self, string=""):
        self.questions[0]['info']['text'] = html(string)
        
    def getname(self):
        return self.questions[0]['category']['text'][len('$module$/top/'):] # removes the $module$/top/
        
    def append(self, question): # adds a Question to a Category
        self.questions.append(question.dict)
                
    def save(self, file_name=None):
        """ Save a category under the format Moodle XML """
        if file_name is None:
            file_name = self.getname()
        category_xml = xmltodict.unparse(self.dict, pretty=True)
        savestr(unescape(category_xml), file_name + ".xml")

### Exemple : Générer une question dans une catégorie et les envoyer dans Moodle

In [94]:
category = Category("ma super catégorie")

In [95]:
question = Question("essay") # creates the question
question.text("Calculer la dérivée de $f(x) = e^x + \frac{1}{2} \Vert x \Vert^2$.") # adds the question text
question.title("Question de cours (dérivée)") # optional
question.graderinfo("$e^x + x$") # optional
question.addto(category)

In [96]:
question = Question("essay")
question.text("Combien font $2+2$?")
question.title("Question d'arithmétique")
question.graderinfo("$\sqrt{16}$")
question.addto(category)
category.save() # maintenant on a deux questions dans la catégorie

La dernière commande crée un fichier `ma_categorie.xml` qu'il suffit d'importer dans Moodle.
Dans l'interface du cours, à droite, aller dans `banque de questions > importer ` puis 
- format de fichier : Format XML Moodle
- Généraux : Cocher "Obtenir la catégorie à partir du fichier", "Obtenir le contexte à partir du fichier" si ce n'est déjà fait
- Glisser le fichier
- Cliquer "importation"
- Si tout est bon (vert) valider encore (continuer).
Ensuite les questions sont prêtes à l'utilisation dans tous les tests.


### Exemple : Générer plein de questions aléatoirement et les envoyer dans Moodle

Ici on montre ce qu'il est possible de faire, évidemment chacun doit adapter à ses besoins.

In [12]:
import numpy as np
from numpy.random import randint
import array_to_latex as a2l

In [13]:
# On veut générer des matrices entières aléatoires et récupérer leur code latex
def strlatex(A): 
    return a2l.to_ltx(A, frmt = '{:2.0f}', arraytype = 'pmatrix', print_out=False)
A = randint(-6,6,(3,3))
print(strlatex(A))

\begin{pmatrix}
   3  & -2  & -2 \\
 -6  & -6  & -2 \\
 -1  & -1  & -1 
\end{pmatrix}


In [14]:
# du coup on peut en mettre plein les questions
category = Category("produit matriciel")
category.description("liste de questions sur le produit matriciel")

for k in range(5):
    A = randint(-6,6,(3,3))
    B = randint(-6,6,(3,2))
    latex = "Calculer le produit matriciel suivant : $$" + strlatex(A) + strlatex(B) + " $$"
    question = Question("essay")
    question.text(latex)
    question.name("Produit matriciel")
    question.graderinfo(strlatex(A@B)) # python nous donne la réponse 
    question.addto(category)
    
category.save()

## Exemples pour créer un QCM et l'envoyer dans Moodle

In [15]:
category = Category("QCM matrices positives")
category.description("liste de questions QCM sur les matrices positives")

In [16]:
# une question théorique
question = Question("multichoice") # L'option ici définit le fait que la question est un QCM
question.text("Est-ce que une matrice symétrique est inversible?")
question.name("Matrice symétrique inversible?")

# les réponses possibles
Answer("Vrai", False).addto(question) # Réponse "vrai", qui est fausse
Answer("Faux", True).addto(question) # Réponse "faux", qui est la bonne réponse
question.addto(category) # on a fini donc ajoute la question à la catégorie

category.save() # on exporte la question si on veut vérifier que ça marche

In [17]:
# une question calculatoire aléatoire
question = Question("multichoice")

latex = "Est-ce que la matrice suivante est positive? "
A = randint(-3,3, (2,2))
A = A@A.T # Forcément symétrique positive
latex = latex + strlatex(A)

question.text(latex)
question.name("Matrice symétrique inversible?")

# les réponses possibles
Answer("Vrai", True).addto(question)
Answer("Faux", False).addto(question)
question.addto(category) # on a fini donc ajoute la question à la catégorie, qui en a deux maintenant.

category.save()

In [18]:
# une question calculatoire mais avec des cas générés aléatoirement 
# donc on ne sait pas ce que ça va donner mais python gère pour nous
question = Question("multichoice")

text = "Est-ce que la matrice suivante est définie positive? "
A = randint(-3,3, (2,2))
A = A + A.T # symétrique mais pas positive a priori
text = text + strlatex(A)

def is_pos_def(A): # check si la matrice est définie positive
    M = np.matrix(A)
    if np.all(np.linalg.eigvals(M+M.transpose()) > 0):
        return True
    else:
        return False
boolean = is_pos_def(A)

question.text(text)
question.name("Matrice symétrique DP?")
question.graderinfo(str(boolean))

# les réponses possibles
Answer("Vrai", boolean).addto(question)
Answer("Faux", not boolean).addto(question)
question.addto(category)

category.save()

In [19]:
# une question avec plus de deux réponses
question = Question("multichoice") 
question.text("Est-ce que vous aimez la pizza aux ananas")
question.name("Pizza question")
question.multi_answer() # il y aura plusierus réponses possibles

# les réponses possibles
Answer("Oui", False).addto(question)
Answer("Bof", 20).addto(question)
Answer("Dégueu", 80).addto(question) # la somme des pourcentages doit faire 100, ici il faut cocher deux réponses pour gagner
question.addto(category)

category.save()

Un peu de documentation:

- https://docs.moodle.org/3x/fr/Format_XML_Moodle
- https://docs.moodle.org/3x/fr/Questions
- https://docs.moodle.org/31/en/Embedded_files_repository
- AMC to Moodle : https://github.com/nennigb/amc2moodle
- GIFT to Moodle : https://www.lmspulse.com/2020/gift-format-moodle-quiz-scripting/
- Dict to XML : https://stackoverflow.com/questions/36021526/converting-an-array-dict-to-xml-in-python
    - bon ca marche pas terrible relou avec attribute et plein de trucs automatiques
- Dict to XML : https://github.com/martinblech/xmltodict
    - meilleur controle sur tout
    - par contre l'output est un string avec des tabs? Ok facile a resoudre
    - ne gère pas le CDATA : besoin de https://github.com/martinblech/xmltodict/issues/57 ? Yes
- QCM donne du `<shownumcorrect/>` càd un élément autofermé. Nous on génère un élément vide qui est sensé faire la même chose `<shownumcorrect></shownumcorrect>` cf https://stackoverflow.com/questions/7231902/self-closing-tags-in-xml-files